# Implement Evaluation Data Generation and Evaluation
To further enhance the prompt generation process, incorporate automatic Evaluation Data Generation. 
- By analysing the description provided by the user,  create a set of test cases that serve as evaluation benchmarks for the prompt candidates.
- These test cases simulate various scenarios, enabling users to observe how each prompt performs in different contexts. 
- The generated test cases serve as a starting point, sparking creativity and inspiring additional test cases for comprehensive evaluation.


In [12]:
import os
import sys
from dotenv import load_dotenv,find_dotenv
sys.path.append(os.path.abspath(os.path.join('../utility')))

In [13]:
from langchain_utility import generate_testcase_and_context, data_loader
from rag import create_retriever

In [14]:
# Load OpenAI API key from .env file
load_dotenv(find_dotenv())

True

In [15]:
chunks = data_loader()
retriever = create_retriever(chunks)


2024-01-18 23:31:55,502:logger:data loaded to vector database successfullu


embedded weaviate is already listening on port 8079


/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/langchain_community/embeddings/openai.py:500: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  response = response.dict()
/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/pydantic/main.py:979: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', DeprecationWarning)


In [16]:
# Specify the path to your .txt file
file_path = '../prompts/evaluation-data-generation.txt'

# Open the file in read mode
with open(file_path, 'r') as file:
    # Read the contents of the file
    file_contents = file.read()

In [23]:
question_answers = [{'prompt': "When did OpenAI showcase the capabilities of reinforcement learning algorithms through its 'OpenAI Five' project?",
  'ground_truth': 'OpenAI showcased the capabilities of these reinforcement learning algorithms through its ‘OpenAI Five’ project in 2018.'},
 {'prompt': 'Who founded OpenAI?',
  'ground_truth': 'OpenAI was initially founded in 2015 by Sam Altman, Elon Musk, Ilya Sutskever and Greg Brockman.'},
 {'prompt': 'What is the goal of OpenAI?',
  'ground_truth': "The stated goal of OpenAI is to 'advance digital intelligence in the way that is most likely to benefit humanity as a whole.'"},
 {'prompt': 'What did OpenAI release in 2016?',
  'ground_truth': "OpenAI released 'OpenAI Gym' in 2016, a toolkit for developing and comparing reinforcement learning algorithms."},
 {'prompt': 'What did OpenAI achieve in the early years?',
  'ground_truth': 'OpenAI made significant progress in research in deep learning and reinforcement learning in the early years.'}]

# Extract prompt values and ground truth values into separate lists
questions = [item['prompt'] for item in question_answers]
ground_truths = [[item['ground_truth']] for item in question_answers]

In [24]:
response = generate_testcase_and_context(questions, ground_truths, retriever, file_contents)

2024-01-18 23:34:41,772:logger:data loaded to vector database successfully
/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/langchain_community/embeddings/openai.py:500: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  response = response.dict()
/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/pydantic/main.py:979: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', DeprecationWarning)
/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/langchain_community/chat_models/openai.py:458: PydanticDeprecatedSince20: The `dict` method is depr

In [25]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset = response, 
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

df = result.to_pandas()

evaluating with [context_precision]


/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/tqdm/std.py:580: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  if rate and total else datetime.utcfromtimestamp(0))
  0%|          | 0/1 [00:00<?, ?it/s]Exception in callback <_asyncio.TaskStepMethWrapper object at 0x7f56aba07d60>()
handle: <Handle <_asyncio.TaskStepMethWrapper object at 0x7f56aba07d60>()>
Traceback (most recent call last):
  File "/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/asyncio/events.py", line 84, in _run
    self._context.run(self._callback, *self._args)
RuntimeError: Cannot enter into task <Task pending name='Task-8' coro=<AsyncClient.aclose() running at /home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/httpx/_client.py:1996>> while another task <Task pending name='Task-2' coro=<Kern

100%|██████████| 1/1 [00:06<00:00,  6.36s/it]


evaluating with [context_recall]


/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/tqdm/std.py:580: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  if rate and total else datetime.utcfromtimestamp(0))
  0%|          | 0/1 [00:00<?, ?it/s]Task was destroyed but it is pending!
task: <Task pending name='Task-8' coro=<AsyncClient.aclose() running at /home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/httpx/_client.py:1996>>
/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/pydantic/_internal/_core_utils.py:352: RuntimeWarning: coroutine 'AsyncClient.aclose' was never awaited
  for k, v in schema['fields'].items():
100%|██████████| 1/1 [00:05<00:00,  5.84s/it]


evaluating with [faithfulness]


/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/tqdm/std.py:580: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  if rate and total else datetime.utcfromtimestamp(0))
100%|██████████| 1/1 [00:16<00:00, 16.07s/it]


evaluating with [answer_relevancy]


/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/tqdm/std.py:580: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  if rate and total else datetime.utcfromtimestamp(0))
  0%|          | 0/1 [00:00<?, ?it/s]/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/langchain_community/embeddings/openai.py:500: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  response = response.dict()
/home/babi/miniconda3/envs/tenx_week6/lib/python3.12/site-packages/pydantic/main.py:979: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pyd

In [26]:
df

,question,answer,contexts,ground_truths,context_precision,context_recall,faithfulness,answer_relevancy
0,When did OpenAI showcase the capabilities of r...,OpenAI showcased the capabilities of reinforce...,[OpenAI showcased the capabilities of these re...,[OpenAI showcased the capabilities of these re...,1.0,1.0,1.0,0.940016
1,Who founded OpenAI?,"OpenAI was founded by Sam Altman, Elon Musk, I...",[OpenAI was initially founded in 2015 by Sam A...,[OpenAI was initially founded in 2015 by Sam A...,1.0,1.0,1.0,0.959200
2,What is the goal of OpenAI?,The goal of OpenAI is to advance digital intel...,[OpenAI was initially founded in 2015 by Sam A...,[The stated goal of OpenAI is to 'advance digi...,1.0,1.0,1.0,1.000000
3,What did OpenAI release in 2016?,"OpenAI released 'OpenAI Gym' in 2016, a toolki...",[The early years of OpenAI were marked with ra...,"[OpenAI released 'OpenAI Gym' in 2016, a toolk...",1.0,1.0,1.0,0.899221
4,What did OpenAI achieve in the early years?,"In the early years, OpenAI achieved significan...",[The early years of OpenAI were marked with ra...,[OpenAI made significant progress in research ...,1.0,1.0,1.0,1.000000
